# Multiprocessing zonal calculations

In [12]:
import sys, os, importlib, math, multiprocessing
import rasterio, geojson

import pandas as pd
import geopandas as gpd
import numpy as np

from h3 import h3
from tqdm import tqdm
from shapely.geometry import Polygon

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.ntlMisc as ntl
import GOSTrocks.mapMisc as mapMisc
from GOSTrocks.misc import tPrint

sys.path.append("../src")
import h3_helper

AWS_S3_BUCKET = 'wbg-geography01'
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
h3_level = 6
multiprocess=True
verbose = True
h3_level = 6
data_prefix = "Urbanization"

admin_bounds = "J:/Data/GLOBAL/ADMIN/ADMIN2/HighRes_20230328/shp/WB_GAD_ADM2.shp"
global_urban_file = "J:/Data/GLOBAL/GHSL/SMOD/GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V1_0.tif"
global_pop_file =   "J:/Data/GLOBAL/GHSL/Pop/GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0.tif"

ghs_smod = rasterio.open(global_urban_file)
ghs_pop = rasterio.open(global_pop_file)

RasterioIOError: J:/Data/GLOBAL/GHSL/Pop/GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0.tif: Permission denied

In [7]:
h3_0_list = h3_helper.generate_lvl0_lists(h3_level, return_gdf=True, buffer0=False)

In [15]:
rMisc.zonalStats?

Signature:
rMisc.zonalStats(
    inShp,
    inRaster,
    bandNum=1,
    mask_A=None,
    reProj=False,
    minVal='',
    maxVal='',
    verbose=False,
    rastType='N',
    unqVals=[],
    weighted=False,
    allTouched=False,
    calc_sd=False,
    return_df=False,
)
Docstring:
Run zonal statistics against an input shapefile. Returns array of SUM, MIN, MAX, and MEAN

:param inShp: input geospatial data to summarize raster
:type inShp: string path to file of gpd.GeoDataFrame
:param inRaster: input raster to summarize
:type inRaster: string path to file or rasterio.DatasetReader
:param bandNum: band in raster to analyze, defaults to 1
:type bandNum: int, optional
:param mask_A: mask the raster data using an identical shaped boolean mask, defaults to None
:type mask_A: np.array, optional
:param reProj: whether to reproject data to match, if not, raise a ValueError if CRS mismatch between inShp and inRaster, defaults to False
:type reProj: bool, optional
:param minVal: if defined, will 

In [14]:
# get counts of urban categorties
unq_urban = [11,12,13,21,22,23,30]
processed_list = []
for h3_0_key, hexes in h3_0_list.items():
    break

In [27]:
res

{'s3://wbg-geography01\\Space2Stats/h3_stats_data/GLOBAL/Urbanization/80b9fffffffffff/GHS_SMOD_2020_counts.csv':         c_11  c_12  c_13  c_21  c_22  c_23  c_30
 0          0     0     0     0     0     0     0
 1         38     0     0     0     0     0     0
 2         26     0     0     0     0     0     0
 3         30     0     0     0     0     0     0
 4         39     0     0     0     0     0     0
 ...      ...   ...   ...   ...   ...   ...   ...
 117644    30     0     0     0     0     0     0
 117645    35     0     0     0     0     0     0
 117646    30     0     0     0     0     0     0
 117647    35     0     0     0     0     0     0
 117648     0     0     0     0     0     0     0
 
 [117649 rows x 7 columns]}

In [26]:
filename = 'GHS_SMOD_2020_counts.csv'            
out_s3_key = f'Space2Stats/h3_stats_data/GLOBAL/{data_prefix}/{h3_0_key}/{filename}'
full_path = os.path.join("s3://", AWS_S3_BUCKET, out_s3_key)
'''
try:
    tempPD = pd.read_csv(full_path)
    processed_list.append(filename)
except:

'''
res = run_zonal_cat(hexes, ghs_smod, full_path, unq_vals=unq_urban, buffer0=True, verbose=True)
res.to_csv(
            f"s3://{AWS_S3_BUCKET}/{out_s3_key}",
            index=False,
            storage_options={
                "key": AWS_ACCESS_KEY_ID,
                "secret": AWS_SECRET_ACCESS_KEY,
                "token": AWS_SESSION_TOKEN,
            },
        )

15:16:12	Starting s3://wbg-geography01\Space2Stats/h3_stats_data/GLOBAL/Urbanization/80b9fffffffffff/GHS_SMOD_2020_counts.csv
15:19:32	**** finished s3://wbg-geography01\Space2Stats/h3_stats_data/GLOBAL/Urbanization/80b9fffffffffff/GHS_SMOD_2020_counts.csv


AttributeError: 'dict' object has no attribute 'to_csv'

In [13]:
def multi_processing_single_raster(sel_iso3, inA, h3_level, r_file, urban_file, out_folder, zonal_label):
    out_file = os.path.join(out_folder, f'{sel_iso3}_{zonal_label}_{h3_level}.csv')
    print(out_file)
    if not os.path.exists(out_file):
        zonalC = country_zonal.country_h3_zonal(sel_iso3, selA, "ID", h3_level, out_folder)
        zonal_res_pop = zonalC.zonal_raster_urban(r_file, urban_file)
        zonal_res_pop.to_csv(out_file)
    return(out_file)

In [14]:
multi_processing_single_raster(*arg_list[0], "GHS_POP_2020")

/home/wb411133/projects/Space2Stats/COUNTRIES/ECU/ECU_GHS_POP_2020_6.csv


Generating h3 grid level 6: 100%|██████████| 28/28 [00:00<00:00, 183.50it/s]


(1, 1827, 2113)


'/home/wb411133/projects/Space2Stats/COUNTRIES/ECU/ECU_GHS_POP_2020_6.csv'

In [ ]:
with multiprocessing.Pool(processes=min([70,len(arg_list)])) as pool:
    results = pool.starmap(multi_processing_single_raster, arg_list)

In [ ]:
multi_processing_single_raster(*arg_list[0], "GHS_POP_2020")